In [ ]:
import numpy as np 
data = np.load("emotion_toy_data/emotion_eeg.npy")
print(data.shape)

(154, 224, 1280)


In [ ]:
labels = np.load("emotion_toy_data/emotion_labels.npy")
print(labels.shape)

(154,)


In [14]:
mask = ~(data == 0).any(axis=(1, 2))
filtered_data = data[mask]
filtered_labels = labels[mask]
print(filtered_data.shape)
print(filtered_labels.shape)

(154, 224, 1280)
(154,)


In [15]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
labels_encoded = enc.fit_transform(filtered_labels)
print(labels_encoded.shape)
unique_labels, counts = np.unique(labels_encoded,return_counts=True)
print(np.asarray((unique_labels, counts)).T)

(154,)
[[ 0 11]
 [ 1 11]
 [ 2 11]
 [ 3 11]
 [ 4 11]
 [ 5 11]
 [ 6 11]
 [ 7 11]
 [ 8 11]
 [ 9 11]
 [10 11]
 [11 11]
 [12 11]
 [13 11]]


In [16]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(filtered_data,labels_encoded,test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(138, 224, 1280)
(16, 224, 1280)
(138,)
(16,)


In [17]:
unique_labels, counts = np.unique(y_train,return_counts=True)
print(np.asarray((unique_labels, counts)).T)

[[ 0 10]
 [ 1 10]
 [ 2  8]
 [ 3 10]
 [ 4  9]
 [ 5 11]
 [ 6  9]
 [ 7 11]
 [ 8 10]
 [ 9  9]
 [10 11]
 [11  9]
 [12 10]
 [13 11]]


In [18]:
unique_labels, counts = np.unique(y_test,return_counts=True)
print(np.asarray((unique_labels, counts)).T)

[[ 0  1]
 [ 1  1]
 [ 2  3]
 [ 3  1]
 [ 4  2]
 [ 6  2]
 [ 8  1]
 [ 9  2]
 [11  2]
 [12  1]]


In [ ]:
from classify import Optimizer
from sklearn.ensemble import GradientBoostingClassifier
from classify import classify_sklearn

optimizer = Optimizer(channels=[i for i in range(X_train.shape[1])],classifier=classify_sklearn,sfreq=256,model=GradientBoostingClassifier())
best_params, best_accuracy = optimizer.optimize_hyperparameters(X_train,y_train)
print(best_params)
print(best_accuracy)

In [ ]:
selected_channels = []
desired_features = []
for i, (param, value) in enumerate(best_params.items()):
   if value:
       if param.isdigit():
           selected_channels.append(int(param))
       else:
           desired_features.append(param)
print(selected_channels)
print(desired_features)


In [20]:
from classify import FeatureWrapper
wrapper = FeatureWrapper()
processed_samples_test = []
for i, sample in enumerate(X_test):
    features = wrapper.compute_features(sample,i,256,selected_channels,desired_features=desired_features)
    processed_samples_test.append(features)
processed_samples_test = np.array(processed_samples_test)
print(processed_samples_test.shape)

(16, 224, 3)


In [21]:
processed_samples = []
wrapper = FeatureWrapper()
for i, sample in enumerate(X_train):
    features = wrapper.compute_features(sample,i,256,selected_channels,desired_features=desired_features)
    processed_samples.append(features)
processed_samples = np.array(processed_samples)
print(processed_samples.shape)

(138, 224, 3)


In [22]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = GradientBoostingClassifier()
X_train = np.reshape(processed_samples,(processed_samples.shape[0],-1))
X_test = np.reshape(processed_samples_test,(processed_samples_test.shape[0],-1))
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred,average='weighted')
recall = recall_score(y_test,y_pred,average='weighted')
f1 = f1_score(y_test,y_pred,average='weighted')
print(f"Accuracy: {accuracy}")
print(f"Precision: {accuracy}")
print(f"Recall: {accuracy}")
print(f"F1: {accuracy}")



Accuracy: 0.625
Precision: 0.625
Recall: 0.625
F1: 0.625


c:\Users\matth\anaconda3\envs\neurotech\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matth\anaconda3\envs\neurotech\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
